In [45]:
# coding:utf-8

import os
import shutil
import json

# 코드를 실행하기 전에 이미지-json 파일이 정확한 쌍을 이루는지 먼저 확인해주세요.

# 원본 annotation 파일들이 있는 경로
annoDir = "/home/boo/Desktop/cocoformat/train/label_3/resampling_label/"
# image가 있는 경로
imageDir = "/home/boo/Desktop/cocoformat/train/image_3/image/"

# anno_dir 내 annotation 파일 이름 리스트업
annoList = os.listdir(annoDir)
print(len(annoList))
print(annoList)


23469
['22_111229_220623_25.json', '08_150721_220615_03.json', '08_150721_220615_32.json', '22_100653_221020_25.json', '08_105027_220627_10.json', '08_144931_220613_18.json', '08_143736_221012_40.json', '21_104728_220620_25.json', '08_193143_220608_04.json', '08_193837_220628_25.json', '08_145803_220609_05.json', '21_093909_220624_08.json', '08_104912_220613_03.json', '22_084101_221020_18.json', '21_104845_220623_37.json', '22_083901_221020_33.json', '22_102303_221020_03.json', '22_082526_220624_18.json', '08_141139_221012_14.json', '08_191632_220628_09.json', '21_082953_220624_02.json', '21_110502_220623_09.json', '21_103431_220622_19.json', '08_154903_220609_10.json', '21_193630_220621_21.json', '08_195042_220628_12.json', '22_070242_221019_21.json', '22_082846_220624_01.json', '21_144446_220829_38.json', '08_105412_220613_26.json', '18_160250_220816_12.json', '08_150906_220613_17.json', '22_072358_221020_32.json', '08_150913_220609_02.json', '22_084301_221020_34.json', '08_194902_22

In [46]:
# anno_dict_resample = {"pedestrian":0,"trafficLight":0,"trafficSign":0, "twoWheeler":0, "vehicle":0}
def categoryid(cls):
    if cls == "pedestrian":
        return 1
    elif cls == "trafficLight":
        return 2  
    elif cls == "trafficSign":
        return 3
    elif cls == "twoWheeler":
        return 4
    elif cls == "vehicle":
        return 5

def id(str) :
    return int(str.replace("_", "").replace(".jpg", ""))

def iscrowd(dic) :
    if "group" in dic:
        return dic["group"]
    else:
        return 0

In [47]:
# cocoDict에 필요한 images, annotations 정보들의 리스트
imgTmpDict = []
annoTmpDict = []

idNum = 1
annoIdNum = 60000
for anno in annoList:
    with open(annoDir + anno, encoding="utf-8-sig", errors="ignore") as json_file:
        jsonData = json.load(json_file)
    
    imgPath = anno.replace(".json", ".jpg")

    imgSize = 1920 * 1080

    try : 
        images = {
            "license": 1,
            "file_name": jsonData["information"]["filename"],
            "coco_url": "",
            "height": jsonData["information"]["resolution"][1],
            "width": jsonData["information"]["resolution"][0],
            "date_captured": jsonData["Environment_meta"]["date"]+" "+jsonData["Environment_meta"]["time"],
            "flickr_url": "",
            "id": id(jsonData["information"]["filename"])
            #"id": idNum
        }
    except :
        images = {
            "license": 1,
            "file_name": jsonData["information"]["filename"],
            "coco_url": "",
            "width": 1920,
            "height": 1080,
            "date_captured": jsonData["Environment_meta"]["date"],
            "flickr_url": "",
            "id": id(jsonData["information"]["filename"])
            #"id": idNum
        }
    imgTmpDict.append(images)

    for i in range(len(jsonData["annotations"])):
        try: # bbox 정보가 존재할 경우
            bbox = [
                jsonData["annotations"][i]["bbox"][0],
                jsonData["annotations"][i]["bbox"][1],
                jsonData["annotations"][i]["bbox"][2]-jsonData["annotations"][i]["bbox"][0],
                jsonData["annotations"][i]["bbox"][3]-jsonData["annotations"][i]["bbox"][1]
            ]
            annotation = {
                "segmentation": [],
                "area": 0,
                "iscrowd": iscrowd(jsonData["annotations"][i]["attribute"]),
                "image_id": id(jsonData["information"]["filename"]),
                #"image_id" : idNum,
                "bbox": bbox,
                "category_id": categoryid(jsonData["annotations"][i]["class"]),
                "id": annoIdNum,
                "flags": [jsonData["annotations"][i]["attribute"]["occluded"], jsonData["annotations"][i]["attribute"]["truncated"]],
            }
        except: # polygon 정보가 존재할 경우
            annotation = {
                "segmentation": [],
                "area": 0,
                "iscrowd": iscrowd(jsonData["annotations"][i]["attribute"]),
                "image_id": id(jsonData["information"]["filename"]),
                #"image_id" : idNum,
                "bbox": [0,0,0,0],
                "category_id": categoryid(jsonData["annotations"][i]["class"]),
                "id": annoIdNum,
                "flags": [jsonData["annotations"][i]["attribute"]["occluded"], jsonData["annotations"][i]["attribute"]["truncated"]],
           }
        annoTmpDict.append(annotation)
        annoIdNum += 1
    idNum += 1


In [48]:
# COCO format annotation이 저장될 dict
cocoDict = {}

cocoDict["info"] = {
  "description": "승용 자율주행차 주간 도심도로 데이터",
  "url": "https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71621",
  "version": "1.1",
  "year": 2024,
  "contributor": "Jiyong Boo",
  "date_created": "2024/04/03",
}

cocoDict["licenses"] = [
  {"url": "https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71621", "id": 1, "name": "㈜인피닉"}
]

cocoDict["images"] = imgTmpDict
cocoDict["annotations"] = annoTmpDict

cocoDict["categories"] = [
  {"supercategory": "S2", "id": 1, "name": "pedestrian"},
  {"supercategory": "S2", "id": 2, "name": "trafficLight"},
  {"supercategory": "S2", "id": 3, "name": "trafficSign"},
  {"supercategory": "S2", "id": 4, "name": "twoWheeler"},
  {"supercategory": "S2", "id": 5, "name": "vehicle"},
]

In [49]:
with open("/home/boo/Desktop/cocoformat/train/5class_coco.json", 'w') as f :
    json.dump(cocoDict, f, ensure_ascii=False)
    print("완료했습니다.")

#with open("/home/boo/Desktop/cocoformat/train/5class_coco_test.json", 'w') as f :
#    json.dump(cocoDict, f, ensure_ascii=False)
#    print("완료했습니다.")

완료했습니다.
